In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


In [230]:
# read all the worksheets form the data excel file
df = pd.read_excel('data/Muesli Project raw data - group 3.xlsx', header=1)
df1 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=1)
df2 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=2)
df3 = pd.read_excel('data/Muesli Project raw data - group 3.xlsx',sheet_name=3)

In [209]:
# check the df headers
print(df.columns)
print(df1.columns)
print(df2.columns)
print(df3.columns)

Index(['Index', 'Order ID', 'Order Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Origin Channel', 'Country/Region', 'City', 'State',
       'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product ID',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date', 'Customer Name'], dtype='object')
Index(['Row ID', 'Order ID', 'Order Date', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


In [231]:
# delete unneeded columns
df = df.drop(['Index','Origin Channel', 'Category', 'Sub-Category'], axis=1)
df1 = df1.drop(['Customer Name'], axis=1)
df2 = df2.drop(['Row ID', 'Order Date'], axis=1)

In [248]:
# check the number of rows and columns
print('df=',df.shape)
print('df1=',df1.shape)
print('df2=',df2.shape)
print('df3=',df3.shape)

df= (9994, 15)
df1= (333, 2)
df2= (5899, 3)
df3= (290, 3)


In [242]:
# check for duplicates
print('df=',df.duplicated().value_counts())
print('-' * 30)
print('df1=',df1.duplicated().value_counts())
print('-' * 30)
print('df2=',df2.duplicated().value_counts())
print('-' * 30)
print('df3=',df3.duplicated().value_counts())

df= False    9993
True        1
Name: count, dtype: int64
------------------------------
df1= False    333
Name: count, dtype: int64
------------------------------
df2= False    3003
True     2896
Name: count, dtype: int64
------------------------------
df3= False    204
True      86
Name: count, dtype: int64


In [244]:
# drop the duplicates
df_no_dups = df.drop_duplicates()
df2_no_dups = df2.drop_duplicates()
df3_no_dups = df3.drop_duplicates()

In [254]:
# check the number of rows and columns
print('df_no_dups=', df_no_dups.shape)
print('df1=', df1.shape)
print('df2_no_dups=', df2_no_dups.shape)
print('df3_no_dups=', df3_no_dups.shape)

df_no_dups= (9993, 15)
df1= (333, 2)
df2_no_dups= (3003, 3)
df3_no_dups= (204, 3)


In [259]:
# check the df headers and decide what to merge with what
print(df_no_dups.columns)
print(df1.columns)
print(df2_no_dups.columns)
print(df3_no_dups.columns)

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')
Index(['Order ID', 'Arrival Scan Date'], dtype='object')
Index(['Order ID', 'On Truck Scan Date', 'Ship Mode'], dtype='object')
Index(['Order ID', 'Ready to Ship Date', 'Pickup Date'], dtype='object')


We have to calculate and validate the company assumptions:
1. Preparation process duration from order reception to shipping (2 days): 'Order Date' - 'Ready to Ship Day' (merge df and df3)
2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3
3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day
4. From "Delivery" to "Order Delivered"

...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)



1. preparation process duration verification

In [313]:
# merge df and df3

df_prep_dur_ver = pd.merge(df_no_dups, df3_no_dups, on='Order ID', how='inner')

In [314]:
# verify the headers of the merged dataframe

df_prep_dur_ver.columns

Index(['Order ID', 'Order Date', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Country/Region', 'City', 'State', 'Postal Code', 'Region',
       'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Ready to Ship Date', 'Pickup Date'],
      dtype='object')

In [315]:
# delete unneeded columns

df_prep_dur_ver = df_prep_dur_ver.drop(['Country/Region', 'City', 'State', 
                                        'Postal Code', 'Region', 'Product ID', 
                                        'Sales', 'Quantity', 'Discount', 'Profit'], axis=1)

In [316]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Ship Mode,Customer ID,Customer Name,Ready to Ship Date,Pickup Date
330,CA-2020-142090,2020-11-30,Standard Class,SC-20380,Shahid Collister,2020-12-07,2020-12-09
28,CA-2020-102610,2020-11-04,Standard Class,CA-12265,Christina Anderson,2020-11-09,2020-11-12
261,CA-2019-163202,2019-09-27,Standard Class,BM-11650,Brian Moss,2019-10-02,2019-10-04
139,CA-2020-122700,2020-11-26,Standard Class,LT-17110,Liz Thompson,2020-11-30,2020-12-02
396,CA-2019-129630,2019-09-04,First Class,IM-15055,Ionia McGrath,2019-09-04,2019-09-04
65,CA-2020-150959,2020-11-11,First Class,TD-20995,Tamara Dahlen,2020-11-13,2020-11-13
340,CA-2019-102981,2019-09-06,Second Class,MO-17500,Mary O'Rourke,2019-09-09,2019-09-11
310,CA-2020-123701,2020-11-24,First Class,PG-18820,Patrick Gardner,2020-11-27,2020-11-27
60,CA-2019-106950,2019-09-02,Standard Class,JE-15715,Joe Elijah,2019-09-06,2019-09-09
162,US-2020-118598,2020-11-16,First Class,CM-12190,Charlotte Melton,2020-11-16,2020-11-16


In [317]:
# check the type of data in the columns

df_prep_dur_ver.dtypes

Order ID                      object
Order Date            datetime64[ns]
Ship Mode                     object
Customer ID                   object
Customer Name                 object
Ready to Ship Date    datetime64[ns]
Pickup Date           datetime64[ns]
dtype: object

In [318]:
# calculate the preparation duration and print it in a new column

df_prep_dur_ver['prep duration'] = df_prep_dur_ver['Ready to Ship Date']-df_prep_dur_ver['Order Date']

In [319]:
# verify

df_prep_dur_ver.sample(10)

,Order ID,Order Date,Ship Mode,Customer ID,Customer Name,Ready to Ship Date,Pickup Date,prep duration
58,CA-2019-149783,2019-09-04,Second Class,DL-13315,Delfina Latchford,2019-09-09,2019-09-11,5 days
289,CA-2020-102925,2020-11-05,Second Class,CD-12280,Christina DeMoss,2020-11-10,2020-11-12,5 days
250,CA-2020-153654,2020-11-24,Second Class,MD-17860,Michael Dominguez,2020-11-26,2020-11-27,2 days
149,CA-2020-114258,2020-11-05,Second Class,EM-13825,Elizabeth Moffitt,2020-11-10,2020-11-12,5 days
292,CA-2020-121818,2020-11-20,First Class,JH-15430,Jennifer Halladay,2020-11-23,2020-11-23,3 days
356,US-2020-110576,2020-11-28,Standard Class,RB-19795,Ross Baird,2020-12-04,2020-12-07,6 days
253,US-2020-136707,2020-11-12,First Class,JE-15610,Jim Epp,2020-11-16,2020-11-16,4 days
354,CA-2020-135307,2020-11-26,First Class,LS-17245,Lynn Smith,2020-11-27,2020-11-27,1 days
321,US-2020-145597,2020-11-02,First Class,GG-14650,Greg Guthrie,2020-11-05,2020-11-06,3 days
247,CA-2019-116540,2019-09-02,First Class,SS-20590,Sonia Sunley,2019-09-02,2019-09-03,0 days


In [320]:
# check thet data type

df_prep_dur_ver['prep duration'].dtype

dtype('<m8[ns]')

what?...

dtype('<m8[ns]') indicates that the data type of the object is a datetime64, representing date and time information with nanosecond precision.

ok, now I get it!

In [321]:
# calculate the mean prep duration
mean_duration_days = df_prep_dur_ver['prep duration'].mean()

# Round the mean duration to the nearest days and hours
rounded_duration = mean_duration_days.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_days = rounded_duration.days
rounded_hours = rounded_duration.seconds // 3600  # Convert seconds to hours

print(f"{rounded_days} days {rounded_hours} hours")

4 days 4 hours


that's way more than the claimed preparation duration!

2. Duration of process from "Ready to Ship" to "on Truck" (1 day for normal, 0 days for express): merge df2 and df3


In [323]:
# merge df2 and df3

df_dur_readytoship_truck = pd.merge(df2_no_dups, df3_no_dups, on='Order ID', how='inner')

In [309]:
# verify the merged dataframe

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date
47,CA-2019-168557,2019-09-25,Standard Processing,2019-09-23,2019-09-25
128,CA-2020-122644,2020-11-18,Standard Processing,2020-11-17,2020-11-18
123,CA-2020-104640,2020-11-18,Standard Processing,2020-11-16,2020-11-18
181,CA-2020-155159,2020-12-02,Standard Processing,2020-11-30,2020-12-02
74,CA-2020-153339,2020-11-06,Standard Processing,2020-11-05,2020-11-06
72,US-2020-163790,2020-11-06,Standard Processing,2020-11-04,2020-11-06
120,CA-2020-155621,2020-11-18,Standard Processing,2020-11-16,2020-11-18
9,CA-2019-106950,2019-09-09,Standard Processing,2019-09-06,2019-09-09
151,CA-2020-157448,2020-11-23,Standard Processing,2020-11-20,2020-11-23
179,CA-2020-119746,2020-11-30,Standard Processing,2020-11-27,2020-11-30


In [325]:
# calculate the duration of the process from "Ready to Ship" to "on Truck"  and print it in a new column

df_dur_readytoship_truck['ship to truck'] = df_dur_readytoship_truck['On Truck Scan Date']-df_dur_readytoship_truck['Ready to Ship Date']

In [327]:
# verify

df_dur_readytoship_truck.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
30,CA-2019-121223,2019-09-16,Standard Processing,2019-09-13,2019-09-16,3 days
25,CA-2019-149223,2019-09-13,Standard Processing,2019-09-11,2019-09-13,2 days
41,CA-2019-158610,2019-09-23,Express,2019-09-23,2019-09-23,0 days
6,CA-2019-106278,2019-09-06,Standard Processing,2019-09-05,2019-09-06,1 days
155,CA-2020-104864,2020-11-25,Standard Processing,2020-11-23,2020-11-25,2 days
77,CA-2020-163188,2020-11-09,Express,2020-11-09,2020-11-09,0 days
110,US-2020-118598,2020-11-16,Express,2020-11-16,2020-11-16,0 days
65,CA-2020-121160,2020-11-04,Express,2020-11-04,2020-11-04,0 days
63,CA-2019-106243,2019-10-07,Standard Processing,2019-10-04,2019-10-07,3 days
162,CA-2020-118122,2020-11-25,Standard Processing,2020-11-24,2020-11-25,1 days


In [328]:
# check the type of data in the columns

df_dur_readytoship_truck.dtypes

Order ID                       object
On Truck Scan Date     datetime64[ns]
Ship Mode                      object
Ready to Ship Date     datetime64[ns]
Pickup Date            datetime64[ns]
ship to truck         timedelta64[ns]
dtype: object

In [340]:
# filter for the standard processing

df_dur_readytoship_truck_standard = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Standard Processing']

In [341]:
# verify

df_dur_readytoship_truck_standard.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
49,CA-2019-145611,2019-09-25,Standard Processing,2019-09-24,2019-09-25,1 days
94,CA-2020-159135,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
138,CA-2020-130386,2020-11-20,Standard Processing,2020-11-18,2020-11-20,2 days
95,CA-2020-157196,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
12,US-2019-147711,2019-09-11,Standard Processing,2019-09-09,2019-09-11,2 days
50,CA-2019-126284,2019-09-25,Standard Processing,2019-09-24,2019-09-25,1 days
29,CA-2019-112893,2019-09-16,Standard Processing,2019-09-13,2019-09-16,3 days
93,US-2020-122672,2020-11-12,Standard Processing,2020-11-09,2020-11-12,3 days
184,CA-2020-119004,2020-12-02,Standard Processing,2020-11-30,2020-12-02,2 days
20,CA-2019-131205,2019-09-11,Standard Processing,2019-09-09,2019-09-11,2 days


In [343]:
# calculate the mean prep duration for the standard processing
mean_to_truck_days_standard = df_dur_readytoship_truck_standard['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_standard = mean_to_truck_days_standard.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_standard = rounded_to_truck_dur_standard.days
rounded_to_truck_hours_standard = rounded_to_truck_dur_standard.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_standard} days {rounded_to_truck_hours_standard} hours")

2 days 0 hours


...that's more than the claimed one day

3. Efficacy of "Express Processing": the order ought to be on the truck on the ready to ship day


In [345]:
# filter for the express processing

df_dur_readytoship_truck_xpress = df_dur_readytoship_truck[df_dur_readytoship_truck['Ship Mode']== 'Express']

In [346]:
# verify

df_dur_readytoship_truck_xpress.sample(10)

,Order ID,On Truck Scan Date,Ship Mode,Ready to Ship Date,Pickup Date,ship to truck
149,CA-2020-157420,2020-11-23,Express,2020-11-23,2020-11-23,0 days
110,US-2020-118598,2020-11-16,Express,2020-11-16,2020-11-16,0 days
3,CA-2019-158099,2019-09-06,Express,2019-09-05,2019-09-06,1 days
5,US-2019-101616,2019-09-06,Express,2019-09-06,2019-09-06,0 days
77,CA-2020-163188,2020-11-09,Express,2020-11-09,2020-11-09,0 days
23,CA-2019-143476,2019-09-13,Express,2019-09-13,2019-09-13,0 days
28,CA-2019-153157,2019-09-16,Express,2019-09-16,2019-09-16,0 days
148,CA-2020-111556,2020-11-23,Express,2020-11-23,2020-11-23,0 days
98,US-2020-116491,2020-11-13,Express,2020-11-13,2020-11-13,0 days
70,US-2020-153948,2020-11-06,Express,2020-11-06,2020-11-06,0 days


In [347]:
# calculate the mean prep duration for the express processing
mean_to_truck_days_xpress = df_dur_readytoship_truck_xpress['ship to truck'].mean()

# Round the mean duration to the nearest days and hours
rounded_to_truck_dur_xpress = mean_to_truck_days_xpress.round('h')  # Round to the nearest hour

# Convert the rounded duration to days and hours
rounded_to_truck_days_xpress = rounded_to_truck_dur_xpress.days
rounded_to_truck_hours_xpress = rounded_to_truck_dur_xpress.seconds // 3600  # Convert seconds to hours

print(f"{rounded_to_truck_days_xpress} days {rounded_to_truck_hours_xpress} hours")

0 days 10 hours


no comments

4. From "Delivery" to "Order Delivered"



...and get image of the whole delivery process duration: from 'Order Date' to 'Arrival Scan Date' (merge df and df2)

In [214]:
#merge the dataframes
merged_df = pd.merge(df, df1, on='Order ID', how='inner')
merged_df1 = pd.merge(merged_df, df2_red_no_dups, on='Order ID', how='inner')
merged_df2 = pd.merge(merged_df1, df3_no_dups, on='Order ID', how='inner')

In [219]:
# merge the dataframes with the 'reduce' function
dfs = [df, df1, df2_red_no_dups, df3_no_dups]

merged_df_red = reduce(lambda left, right: pd.merge(left, right, on='Order ID', how='inner'), dfs)

merged_df_red.shape

(41, 26)